In [1]:
from sklearn.feature_selection import RFECV, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import Imputer


from sklearn.metrics import roc_auc_score
from sklearn import linear_model

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pandas as pd
import numpy as np
import math
import os
import time

import load_data
import file_structure

from train_helper import RF_grid, RF_random_grid, Logreg_grid

def label_conversion(numpy_array):
    labels = numpy_array.tolist()
    labels_str = list()

    for i in range(len(labels)):

        if numpy_array[i] == 0:
            labels_str.append('non-codel')
        else:
            labels_str.append('codel')

    return labels_str


In [2]:

# modals
modal_list = ['FLAIR','T1', 'T1post','T2']
#study = 'CNN' # <== CHANGE HERE
# study_list = ['texture','TDA','CNN']
study_list = ['texture','TDA','CNN','cmb']
ICC_filt = 3

for ICC_filt in (4,3,1):
    if ICC_filt == 1:
        suffix = '_icc2'
    elif ICC_filt >= 3:
        suffix = '_icc%i' % ICC_filt
    else: 
        suffix = '' 

    for study in study_list:
        (origdir,basedir,imagedir,normdir,splitdir,out_dir,model_dir) = file_structure.file_dirs(study)
        # load feature names and corr+rfe filtered feature names
        #if study == 'TDA':
        #    feature_names = np.load(data_dir + 'ph_feature_names.npy')
        #    corr_rfe_selected_list = np.load(os.path.join(out_dir,'corr_rfe_ph_features_09.npy')).item()
        #elif study == 'texture':
        #    feature_names = np.load(data_dir + 'rfecv_tex_features_selected.npy')
        #    corr_rfe_selected_list = np.load(os.path.join(out_dir, 'rfe_tex_features_selected.npy')).item()
        corr_rfe_selected_list = np.load(os.path.join(out_dir,'rfe_%s_features_selected%s.npy' % (study.lower()[0:3],suffix))).item()
        random_state_list = [2,4,6,8,10,12,14,16,18,20] 

        # For timing purpose
        time_list = []

        for fold in range(10):
            print('%s fold number: %d' % (study, fold))
            #t0 = time.time() python 2 vs python 3
            t0 = time.time()

            random_state = random_state_list[fold]

            fold_dir = os.path.join(splitdir,'fold_' + str(fold))

            #study_saved_model = data_dir_fold + 'saved_models/' + study + '_saved_model/'

            # load data
            (train_samples, train_features_pca, train_1p19q, train_age, 
             train_KPS, train_gender, train_id, train_fnames)= load_data.load_data('train',fold,study,ICC_filt)
            (val_samples,val_features_pca, val_1p19q, val_age, 
             val_KPS, val_gender, val_id, val_fnames)= load_data.load_data('val',fold,study,ICC_filt)

            feature_names = train_fnames

            # conver 1,0 into string: 'codel' and 'non-codel'
            train_1p19q_str = label_conversion(train_1p19q)
            val_1p19q_str = label_conversion(val_1p19q)

            # Create clinical variable df
            clinical_names = ['age', 'KPS', 'gender']
            train_clinical_df = pd.DataFrame(data = np.vstack((train_age, train_KPS, train_gender)).transpose(), 
                                                columns = clinical_names, index=train_1p19q_str)
            val_clinical_df = pd.DataFrame(data = np.vstack((val_age, val_KPS, val_gender)).transpose(), 
                                                columns = clinical_names, index=val_1p19q_str)

            # leave one group out for CV purpose later
            logo = LeaveOneGroupOut()

            for train_features, val_features, rfe_pca_flag in [(train_features_pca, val_features_pca,'pca'),
                                                               (train_clinical_df, val_clinical_df,'clinical'),
                                                               (train_samples, val_samples,'rfe')
                                                              ]:

                if rfe_pca_flag == 'rfe': # original features (i.e. non PCA features). This part needs to be filtered with corr_rfe selection
                    # create the pandas dataframes
                    train_features_df = pd.DataFrame(data=train_features, columns=feature_names, index=train_1p19q_str)
                    val_features_df = pd.DataFrame(data=val_features, columns=feature_names, index=val_1p19q_str)

                    # Remove Flatness and LeastAxis since they are all 0s for every sample
                    drop_features = ['shape_Flatness_', 'shape_LeastAxis_']
                    drop_features = [elemt + modal for modal in modal_list for elemt in drop_features]

                    train_features_df = train_features_df.drop(drop_features, axis=1, errors = 'ignore')
                    val_features_df = val_features_df.drop(drop_features, axis=1, errors = 'ignore')

                    # obtain feature names to select from corr + RFE
                    corr_rfe_selected = corr_rfe_selected_list[fold]

                    # only the corr rfe selected features
                    train_features_df = train_features_df[corr_rfe_selected]
                    val_features_df = val_features_df[corr_rfe_selected]

                elif rfe_pca_flag == 'pca': #PCA features
                    # create the pandas dataframes
                    train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                    val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)
                elif rfe_pca_flag == 'clinical': #PCA features
                    # create the pandas dataframes
                    train_features_df = pd.DataFrame(data=train_features, index=train_1p19q_str)
                    val_features_df = pd.DataFrame(data=val_features, index=val_1p19q_str)

                # combine train and val for CV purpose
                train_val_features_df = pd.concat([train_features_df, val_features_df], axis=0)
                train_val_clinical_df = pd.concat([train_clinical_df, val_clinical_df], axis=0)
                train_val_1p19q = np.append(train_1p19q, val_1p19q)
                train_val_1p19q_str = train_1p19q_str + val_1p19q_str

                # combine the membership as well for CV purpose
                #train_val_group_membership = train_group_membership + val_group_membership
                train_val_id = np.append(train_id, val_id)

                # features + clinical 
                train_val_all_df = pd.concat([train_val_features_df, train_val_clinical_df], axis=1)



                for sample_case in ['features', 'all']: #features without clinical vs features + clinical
                    # group kfold split (initially unused, donnie did leave one out)
                    # cross validation groups were too small, and it was causing errors
                    #if sample_case == 'features':
                    #elif sample_case == 'all':
                    if (rfe_pca_flag == 'clinical' and sample_case != 'features'):
                        continue # only run clinical once through

                    for classifier in ['rf','logreg']:
                        print(sample_case,classifier,rfe_pca_flag)
                        if classifier == 'rf':
                            estimator_instance = RandomForestClassifier(random_state=(random_state*2)) 
                            estimator_grid = RF_random_grid()

                            n_iter = 100
                            if sample_case == 'features':
                                gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                                gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)

                                estimator = RandomizedSearchCV(
                                    estimator = estimator_instance, 
                                    param_distributions = estimator_grid,
                                    scoring = 'roc_auc',
                                    #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                    cv = gkfSp,
                                    verbose=2,
                                    random_state = random_state,
                                    n_iter = n_iter,
                                    refit = True,
                                    n_jobs = -1)

                                estimator.fit(train_val_features_df, train_val_1p19q)

                            elif sample_case == 'all':
                                gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                                gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                                estimator = RandomizedSearchCV(
                                    estimator = estimator_instance, 
                                    param_distributions = estimator_grid,
                                    scoring = 'roc_auc',
                                    #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                    cv = gkfSp,
                                    verbose=2,
                                    random_state = random_state,
                                    n_iter = n_iter,
                                    refit = True,
                                    n_jobs = -1)
                                estimator.fit(train_val_all_df, train_val_1p19q)

                        elif classifier == 'logreg':
                            estimator_instance = LogisticRegression(random_state = (random_state*2))
                            estimator_grid = Logreg_grid()

                            if sample_case == 'features':
                                gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                                gkfSp = gkf.split(train_val_features_df,train_val_1p19q, groups = train_val_id)

                                estimator = GridSearchCV(
                                    estimator = estimator_instance, 
                                    param_grid = estimator_grid,
                                    scoring = 'roc_auc',
                                    #cv = logo.split(train_val_features_df, train_val_1p19q, groups = train_val_id),
                                    cv = gkfSp,
                                    verbose=2,
                                    refit = True,
                                    n_jobs = -1)

                                estimator.fit(train_val_features_df, train_val_1p19q)

                            elif sample_case == 'all':
                                gkf = GroupKFold(n_splits=5) #5-fold cross-validation
                                gkfSp = gkf.split(train_val_all_df,train_val_1p19q, groups = train_val_id)
                                estimator = GridSearchCV(
                                    estimator = estimator_instance, 
                                    param_grid = estimator_grid,
                                    scoring = 'roc_auc',
                                    #cv = logo.split(train_val_all_df, train_val_1p19q, groups = train_val_id),
                                    cv = gkfSp,
                                    verbose=2,
                                    refit = True,
                                    n_jobs = -1)

                                estimator.fit(train_val_all_df, train_val_1p19q)

                        best_estimator = estimator.best_estimator_.__dict__
                        best_score = estimator.best_score_

                        # save models and train score
                        estimator_name = ('%s_%s_%s_%s%s.npy' %
                                          (classifier,sample_case,rfe_pca_flag,fold,suffix))
                        estimator_score_name =('%s_%s_%s_%s_best_cv_score%s.npy' %
                                               (classifier,sample_case,rfe_pca_flag,fold,suffix))

                        np.save(os.path.join(model_dir,estimator_name), best_estimator)
                        np.save(os.path.join(model_dir,estimator_score_name), best_score)

            t1 = time.time()
            time_list.append((t1-t0)/60)
            print('%s FOLD: %d DONE WITH TIME TAKEN (IN MIN): %g' % (study, fold, (t1-t0)/60))


texture fold number: 0
features rf pca
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.2min finished


features logreg pca
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  75 out of  90 | elapsed:    3.8s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    3.9s finished


all rf pca
Fitting 5 folds for each of 100 candidates, totalling 500 fits


...

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.9min finished


all logreg rfe
Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.8s


cmb FOLD: 9 DONE WITH TIME TAKEN (IN MIN): 13.2938


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    5.8s finished
